In [17]:
#Useful imports 
import time
time_duration = 0.5
import requests
from bs4 import BeautifulSoup

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

## I/ COLLECTING DATA

In [18]:
#We import selenium driver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--window-size=1920x1080")
options.add_argument("--verbose")
options.add_argument('-no-sandbox')
options.add_argument('-headless')



from selenium.webdriver.common.by import By

driver = webdriver.Chrome(options=options,)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



### I.1/We list all the categories of collects 

In [19]:
#To achieve this, we go on the homepage of gofundme and find the proper soup

r = requests.get('https://www.gofundme.com/fr-fr')
html = r.text
soup = BeautifulSoup(html, 'html.parser')
a = soup.find_all('a',class_='section-categories-icon cell text-center')
Categories = []
for link in a:
    carac = link.get('href')
    if carac[7]=="s":
        Categories.append(carac[13:])
    elif carac[7]== 'd':
        Categories.append(carac[16:])
    #Page_Links.append(link.get('href'))
    #print(link.get('href'))
    #print(a.get('href'))
for i in range(len(Categories)):
    Categories[i] = Categories[i].replace('fundraising','fundraiser')
Categories

#print(len(Page_Links))

['medical-fundraiser',
 'memorial-fundraiser',
 'emergency-fundraiser',
 'charity-fundraiser',
 'education-fundraiser',
 'animal-fundraiser',
 'environment-fundraiser',
 'business-fundraiser',
 'community-fundraiser',
 'competition-fundraiser',
 'creative-fundraiser',
 'event-fundraiser',
 'faith-fundraiser',
 'family-fundraiser',
 'sports-fundraiser',
 'travel-fundraiser',
 'volunteer-fundraiser',
 'wishes-fundraiser']

### I.2.1/ Collecting name, amount collected, url and town using Selenium on each category's homepage </b>

#### We use Selenium and BeautifulSoup in this section because we have to click on a button 'More projects'

In [20]:
data_name = []
data_collect = []
data_links = []
data_town = []

### FOR EACH CATEGORY, WE GO ON ITS HOMEPAGE
for i in range (len(Categories)):
    
    l = "https" + ":"+"//www.gofundme.com/discover/" + Categories[i]
    driver=webdriver.Chrome('chromedriver',options=options)
    driver.get(l)
    
    #Each click on the button "more" add 12 projects, 
    #we click 8 times per categories to have 1728 projects in total
    ButtonMore=driver.find_element(By.CSS_SELECTOR,"div.text-center")
    for i in range(8):
        time.sleep(1.1)
        ButtonMore.click()
   
    

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    
    #AmountCollected and Name 
    a = soup.findAll('div',class_="campaign-tile-img--contain js-lazy")
    collect = soup.findAll('div',class_="show-for-medium")
    for text in a:
        data_name.append(text.get('aria-labelledby'))
    for m in range (1,len(collect)+1,2):
        money = collect[m].get_text()
        data_collect.append(money[1:])
    
    #Links
    Links = soup.findAll('a',class_='fund_tile_card_link')
    for link in Links:
        data_links.append(link.get('href'))

    #Town 
    Town = soup.findAll('div',class_="fund-item fund-location truncate-single-line")
    for town in Town:
        ville = town.find('span',class_="")
        data_town.append(ville.get_text())

        
        
###Cleaning text
for i in range (len(data_collect)):
    data_collect[i] = data_collect[i].replace('\xa0','')

for i in range(len(data_name)):
    data_name[i] = data_name[i].replace('-',' ')   

In [21]:
print(len(data_collect))
print(len(data_links))
print(len(data_name))
print(len(data_town))

1716
1716
1716
1716


###  I.2.2/Collecting Pourcentage of target collected, date creation, category </b>

#### Now that we have the link of each project, we go on each's specific webpage to collect more features with BeautifulSoup

In [ ]:
Cat = []
Pourcentage_Raised = []
Creation_date = []
Description = []
NumbersOfDonors=[]


### FOR EACH PROJECT, WE GO ON ITS OWN PAGE
for e in data_links :
    r = requests.get(e)
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    
    #Category
    a = soup.find('a',class_='m-campaign-byline-type divider-prefix meta-divider flex-container align-center color-dark-gray hrt-tertiary-button hrt-base-button hrt-link hrt-link--gray-dark hrt-link--unstyled')
    if not a:
        Cat.append('None')
    else:
        cat = a.get_text()
        Cat.append(cat)
    
    #Progress
    progress = soup.find('progress',class_='a-progress-bar a-progress-bar--green')
    if not progress:
        Pourcentage_Raised.append("None")
    else:
        pourcentage_funding = progress.get('value')
        pourcentage_funding_float = round(float(pourcentage_funding),1)
        if (pourcentage_funding_float > 100) : 
          pourcentage_funding_float = 100 
        Pourcentage_Raised.append(pourcentage_funding_float)
   
    #Date of Creation
    orga = soup.find('span',class_='m-campaign-byline-created a-created-date')
    if not orga:
        Creation_date.append('None')
    else:
        Creation_date.append(orga.get_text())
    
    #Descritpion
    a = soup.find('div',class_="o-campaign-description")
    description=''
    if not a:
        Description.append('None')
    else:
        for x in a:
            text= x.get_text()
            description += text
        description  = description.replace('\n','')
        description  = description.replace('\xa0','')
        Description.append(description)
    
    #Number of donors 
    a = soup.findAll('script')
    b = str(a)
    indice = b.index('donation_count') 
    i=0
    stringNumber = ''
    while b[indice + 16+i] != ",":
        localNumber = b[indice+16+i]
        stringNumber += localNumber
        i = i+1
    NumbersOfDonors.append(int(stringNumber))

###  I.3.1/Gathering in one DF

In [ ]:
data_final = []
for i in range(len(data_links)):
    data_final.append({
        'title' : data_name[i],
        'Categorie' : Cat[i],
        'collect': data_collect[i],
        'town': data_town[i],
        'Pourcentage Raised' : str(Pourcentage_Raised[i]) + "%",
        'Creation Date' : Creation_date[i],
        'Description' : Description[i],
        'NumberDonors' : NumbersOfDonors[i]
         
        
    })

In [ ]:
df= pd.DataFrame(data_final)
df

### I.3.2/ Export in csv to clean (II) and analyse (III)

In [ ]:
#Export
df.to_csv("DataFinalfromWebScraping2.csv")